# MNIST Training using PyTorch

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

## Data
### Getting the data



In [2]:
from torchvision import datasets, transforms

datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [3]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-407247006381/sagemaker/DEMO-pytorch-mnist


## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [4]:
!pygmentize mnist.py

import argparse
import json
import logging
import os
import sagemaker_containers
import sys
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.


### Use SageMaker Local

Before we launch a SageMaker training job, we can use SageMaker Local to deploy and run our PyTorch training container on our SageMaker Notebook instance. This gives us a means to test our script locally in the training container before we launch a remote training job. We create a local training job by specifying `local` as our instance type.

In [5]:
from sagemaker.pytorch import PyTorch

local_estimator = PyTorch(entry_point='mnist.py',
                    role=role,
                    framework_version='1.2.0',
                    train_instance_count=1,
                    train_instance_type='local',
                    hyperparameters={
                        'epochs': 1
                    })

SageMaker Local allows you to serve the data set from the local file system.

In [6]:
local_estimator.fit({"training": "file://./data"})

Creating tmpca5bv1_algo-1-0hxaz_1 ... 
Attaching to tmpca5bv1_algo-1-0hxaz_12mdone
algo-1-0hxaz_1  | 2020-01-22 06:55:22,220 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
algo-1-0hxaz_1  | 2020-01-22 06:55:22,224 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-0hxaz_1  | 2020-01-22 06:55:22,242 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
algo-1-0hxaz_1  | 2020-01-22 06:55:22,246 sagemaker_pytorch_container.training INFO     Invoking user training script.
algo-1-0hxaz_1  | 2020-01-22 06:55:22,455 sagemaker-containers INFO     Module mnist does not provide a setup.py. 
algo-1-0hxaz_1  | Generating setup.py
algo-1-0hxaz_1  | 2020-01-22 06:55:22,456 sagemaker-containers INFO     Generating setup.cfg
algo-1-0hxaz_1  | 2020-01-22 06:55:22,456 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-0hxaz_1  | 2020-01-22 06:55:22,457 sagemaker-containers INFO     Installi

### Setup Hyperparameter Tuning

We can use SageMaker to automated the hyperparameter search process. SageMaker leverages an algorithm called Bayesian Optimization to automate this process.

Below we define a range and set of values we want SageMaker to search across to discover optimal values for the learning rate and batch-size.

In [8]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {'lr': ContinuousParameter(0.001, 0.1),'batch-size': CategoricalParameter([32,64,128,256,512])}

Next we define the objective metric we seek to optimize. In this case, the objective is to find the hyperparameters that minimizes average loss on our test set. The tuner monitors progress by parsing the training logs. The `metric definitions` describes how to parse the object metric value from the logs.

In [9]:
objective_metric_name = 'average test loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'average test loss',
                       'Regex': 'Test set: Average loss: ([0-9\\.]+)'}]

Next we define a PyTorch estimator. It describes how to execute the training job, the infrastructure and environment. Note that `train_instance_count` is set to two, which will result in distributed training. 'gloo' is selected as the backend for facilitating communication in the distributed training job.

In [10]:
estimator = PyTorch(entry_point='mnist.py',
                    role=role,
                    framework_version='1.2.0',
                    train_instance_count=2,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })

Now, we'll create a HyperparameterTuner object, to which we pass:

* The PyTorch estimator we created above
* Our hyperparameter ranges
* Objective metric name and definition
* Tuning resource configurations such as Number of training jobs to run in total and how many training jobs can be run in parallel.

The code is commented out because the tuner job will require resources and time that is beyond what we have for today's workshop.

In [11]:
#tuner = HyperparameterTuner(estimator,
#                            objective_metric_name,
#                            hyperparameter_ranges,
#                            metric_definitions,
#                            max_jobs=9,
#                            max_parallel_jobs=3,
#                            objective_type=objective_type)

Instead, we'll run a single distributed training job.

After we've constructed our `PyTorch` object, we call the fit method to start teh training job. The data is uploaded from the specified S3 location.

In [12]:
estimator.fit({'training': inputs})

2020-01-22 06:58:36 Starting - Starting the training job...
2020-01-22 06:58:38 Starting - Launching requested ML instances......
2020-01-22 06:59:38 Starting - Preparing the instances for training......
2020-01-22 07:00:56 Downloading - Downloading input data...
2020-01-22 07:01:33 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-01-22 07:01:49,531 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-01-22 07:01:49,534 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-22 07:01:49,546 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-01-22 07:01:49,974 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-01-22 07:01

## Host
### Create endpoint
After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`. Here we will deploy the model to a single ```ml.m4.xlarge``` instance.

Note that we could have tested our model serving script using SageMaker Local. The following line of code will deploy the model serving container on this notebook instance so that you can test the endpoint locally.

`local_predictor = local_estimator.deploy(initial_instance_count=1, instance_type='local')`

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-----------------

### Evaluate
We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a `data` variable in this notebook, which we can then pass to the `predictor`.

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

In [ ]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [ ]:
estimator.delete_endpoint()